# Run RF-OBS+ (Random Forest Observed Plus) model 
December 13, 2022 Soomin Chun @The Ohio State University <br>
This code is to run RF-OBS+ for Maumee River Basins. The model can predict monthly flow, mineral (soluble) phosphorus load, total nitrogen load, total phosphorus load, and sediment loads.

### Import packages and set folder path

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import pickle

from pathlib import Path
# base_path = Path('D:/2023OSU_research/ML_SWAT/Github') # CHANGE FOLDER PATH TO WHERE WANT TO SAVE RESULTS
input_file_path= Path("D:/2023OSU_research/ML_SWAT/Github/inputs") # CHANGE FOLDER PATH TO WHERE WITH INPUT DATA

### Set water quality variables for prediction

In [2]:
# target = 'FLOW_OUTcms' # streamflow
target = 'MINP_OUTkg' # mineral (soluble) phosphorus laod
# target = 'TOT_Nkg' # total nitrogen laod
# target = 'TOT_Pkg' # total phosphorus load
# target = 'SED_OUTtons' # sediment load

### Load input file

In [3]:
# load .csv files for Maumee
WQ_inputs = pd.read_csv(input_file_path/'MAUMEE_SWAT_2005_2021_Monthly_Watershed_AVG.csv')
WQ_inputs['mdate'] = pd.to_datetime(WQ_inputs['mdate'])
WQ_inputs

,mdate,Latitude,Elev_m,Slope,AREAkm2,Corn_frac,Soybean_frac,Wheat_frac,Specialty_frac,Pasture_frac,...,PETmm,TMP_AVC,TMP_MXC,TMP_MNC,SOLAMJ_per_m2,FLOW_OUTcms,MINP_OUTkg,TOT_Nkg,TOT_Pkg,SED_OUTtons
0,2005-01-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,16.853194,-3.964226,0.279063,-8.207693,7.924194,641.20,244400.0,9767000,1343000.0,436700.0
1,2005-02-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,26.034647,-1.465025,2.621159,-5.551222,12.434177,326.80,109300.0,2295000,569300.0,95100.0
2,2005-03-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,62.051445,0.921802,5.879000,-4.035535,16.207334,159.70,33930.0,2146000,119700.0,30390.0
3,2005-04-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,120.414479,10.251565,16.731267,3.771558,19.117697,126.30,17880.0,3513000,37810.0,28960.0
4,2005-05-15,41.9341,245.506824,0.013824,16540,0.325782,0.363957,0.067404,0,0.077254,...,123.610465,13.356957,19.720219,6.993814,22.167183,63.71,13170.0,1867000,22490.0,7747.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2021-08-15,41.9341,245.506824,0.013824,16540,0.318247,0.338138,0.100758,0,0.077254,...,174.785043,23.546380,29.332882,17.760002,21.214173,43.11,7033.0,213400,13350.0,3084.0
200,2021-09-15,41.9341,245.506824,0.013824,16540,0.318247,0.338138,0.100758,0,0.077254,...,138.508638,19.604410,26.000891,13.207961,19.057700,114.80,18330.0,781100,63790.0,33220.0
201,2021-10-15,41.9341,245.506824,0.013824,16540,0.275843,0.242567,0.238733,0,0.077254,...,81.258752,15.039447,19.945669,10.133171,12.503560,383.40,53100.0,3947000,182100.0,172100.0
202,2021-11-15,41.9341,245.506824,0.013824,16540,0.275843,0.242567,0.238733,0,0.077254,...,50.614217,4.287589,9.221812,-0.646469,10.697386,113.30,14640.0,1008000,39750.0,22620.0


### Load RF-OBS+ models

In [4]:
filename = str(input_file_path) + "\\model\\RFOBS_1st_" + target + ".sav"
rf_reg_obs_1st = pickle.load(open(filename, 'rb'))

filename = str(input_file_path) + "\\model\\RFOBS_2nd_" + target + ".sav"
rf_reg_obs_2nd = pickle.load(open(filename, 'rb'))

C:\Users\Soomin\anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Soomin\anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### Run RF-OBS+ models

In [5]:
df = WQ_inputs

# Add prediction from 1st model
df['Predicted']=rf_reg_obs_1st.predict(df[['Corn_frac', 'Soybean_frac', 'Wheat_frac', 'FERTkg_per_ha', 'PRECIPmm', 'PETmm', 'TMP_MXC', 'TMP_MNC']].to_numpy()).tolist()

# Add prediction from 2nd model
df['Predicted']=rf_reg_obs_2nd.predict(df[['Predicted','Frac_Buffer','Frac_Subsurf_Place']].to_numpy()).tolist()

WQ_outputs=df

WQ_outputs.head() # Column 'predicted' is WQ outputs predicted from RF-OBS+ model

,mdate,Latitude,Elev_m,Slope,AREAkm2,Corn_frac,Soybean_frac,Wheat_frac,Specialty_frac,Pasture_frac,...,TMP_AVC,TMP_MXC,TMP_MNC,SOLAMJ_per_m2,FLOW_OUTcms,MINP_OUTkg,TOT_Nkg,TOT_Pkg,SED_OUTtons,Predicted
0,2005-01-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,-3.964226,0.279063,-8.207693,7.924194,641.20,244400.0,9767000,1343000.0,436700.0,128222.493497
1,2005-02-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,-1.465025,2.621159,-5.551222,12.434177,326.80,109300.0,2295000,569300.0,95100.0,49711.802758
2,2005-03-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,0.921802,5.879000,-4.035535,16.207334,159.70,33930.0,2146000,119700.0,30390.0,24008.385197
3,2005-04-15,41.9341,245.506824,0.013824,16540,0.271357,0.280408,0.205379,0,0.077254,...,10.251565,16.731267,3.771558,19.117697,126.30,17880.0,3513000,37810.0,28960.0,46738.498551
4,2005-05-15,41.9341,245.506824,0.013824,16540,0.325782,0.363957,0.067404,0,0.077254,...,13.356957,19.720219,6.993814,22.167183,63.71,13170.0,1867000,22490.0,7747.0,8990.543983
